<a href="https://colab.research.google.com/github/fjpa121197/ImageCLEF2021/blob/colab/Information_Retrieval_Approach/Feature_Extraction_Densenet_121_ImageCLEF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from zipfile import ZipFile
os.environ['KAGGLE_USERNAME'] = "#####" # username from the json file
os.environ['KAGGLE_KEY'] = "#####" # key from the json file
!kaggle datasets download -d fjpa121197/imageclefmed-concept-detection-2021

 90% 130M/145M [00:01<00:00, 114MB/s]
100% 145M/145M [00:01<00:00, 135MB/s]


In [ ]:
# Unzip 2021 data
clef2021 = "/content/imageclefmed-concept-detection-2021.zip"
with ZipFile(clef2021, 'r') as zip:
  zip.extractall()
  print('done with 2021 dataset')

done with 2021 dataset


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm
import csv
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import json

**Run the following cell only if you want to use 2020 dataset**

In [ ]:
# Download 2020 data
!kaggle datasets download -d fjpa121197/imageclefmed2021
# Unzip 2020 data
clef2020 = "/content/imageclefmed2021.zip"
with ZipFile(clef2020, 'r') as zip:
  zip.extractall()
  print('done with 2020 dataset')

100% 3.83G/3.84G [01:15<00:01, 11.3MB/s]
100% 3.84G/3.84G [01:15<00:00, 54.8MB/s]
done with 2020 dataset


# Feature extraction using DenseNet-121 and 2021 dataset

For the feature extraction, three approaches will be used.
*   Use default DenseNet-121 model (imagenet weights) and use it to extract feature of the images without fine-tunning. **DONE!!!!**
*   Use default DenseNet-121 model (imagenet weights) and use it to extract features of the images but using fine-tunning. **DONE!!!**
*   Use a trained DenseNet-121 model (trained to classify 7 modalities using 2020 dataset) and use it to extract features of the images (no fine-tunning). **DONE!!!**
*   Use a trained DenseNet-121 model (trained to classify 7 modalities using 2020 dataset) and use it to extract features of the images but using fine-tunning.


**Extraction of features have previously been done. However, the image loaded was of shape 64x64. This current version is using 224x224. See Trello's board, and search for the feature extraction task, you can find the kaggle api command for the features extracted (64x64).**




## Default DenseNet-121 model (imagenet weights) wo fine-tunning

In [ ]:
default_densenet = tf.keras.applications.densenet.DenseNet121(include_top=False, weights= 'imagenet')
#default_densenet.summary()

In [ ]:
def feedForward(fname, model):
  """
    Function for preprocessing an image and then use a model to predict/extract features of a given image.
    fname: full path and name of the image to be given to the model
    model: model to be used for feature extraction
    
    Returns: a 1-D array that represents the features of the image based on the model.
  """
  img = tf.keras.preprocessing.image.load_img(fname, target_size=(224,224))
  x = tf.keras.preprocessing.image.img_to_array(img)
  x = np.expand_dims(x, axis = 0)
  x = tf.keras.applications.densenet.preprocess_input(x)
  features = model.predict(x)
  features = features.flatten()

  return features

def extract_store(path_dir, model):
  """
    Function that takes a path and passes all the images (inside that directory) to the feedForward function for feature extraction, and then creates
    a dataframe containing all features and image ids of the images.
    path_dir: full path of the directory that will be searched
    model: model to be used for feature extraction

    Returns: A pandas DataFrame containing all images and their corresponding features
  """
  id_features_vector = []
  print('Extracting features...')

  for image in tqdm(os.listdir(path_dir), position= 0, leave = False):
    path = os.path.join(path_dir, image)
    name_image = os.path.splitext(image)[0]
    image_id = int(name_image.replace('synpic',''))
    vector = feedForward(path, model)
    vector = np.insert(vector,0,image_id)
    id_features_vector.append(vector)
  #df_values = pd.DataFrame(id_features_vector, dtype = float)

  return id_features_vector

In [ ]:
# Training set
id_features_vector = extract_store('/content/ImageCLEF2021_ConceptDetection_Training-Set/ImageCLEF2021_ConceptDetection_Training-Set/Training-Images', default_densenet)
#default_features_df.to_csv('/content/features-densenet-default-training-2021.csv', index = False, header=False)
np.save('/content/features-224-densenet-default-training-2021.npy', id_features_vector)


In [ ]:
# Validation set
id_features_vector = extract_store('/content/ImageCLEF2021_ConceptDetection_Validation-Set/Validation-Images', default_densenet)
#default_features_df.to_csv('/content/features-densenet-default-validation-2021.csv', index = False, header=False)
np.save('/content/features-224-densenet-default-validation-2021.npy', id_features_vector)

  0%|          | 2/500 [00:00<00:28, 17.56it/s]

Extracting features...


## Default Densenet-121 model (initialized with imagenet weights) with fine-tunning

In [ ]:
default_densenet = tf.keras.applications.densenet.DenseNet121(include_top=True, weights= 'imagenet')

33193984/33188688 [==============================] - 0s 0us/step


In [ ]:
# Utils functions
def extract_concepts(root_paths, csv_names, image_id_concepts_dict = dict()):

    if len(root_paths)==len(csv_names):
      for idx, name in enumerate(root_paths):
        path = os.path.join(name, csv_names[idx])
        if csv_names[idx] == 'Training_Set_Concepts.csv': # Make sure that you just convert the .xlsx file to csv manually
          with open(path, "r", encoding= 'utf-8-sig') as f:
            reader = csv.reader(f, delimiter = ',')
            for i, line in enumerate(reader):
              image_id_concepts_dict[line[0]+'.jpg'] = list(line[1].split(';'))
        else:
           with open(path, "r", encoding= 'utf-8-sig') as f:
            reader = csv.reader(f, delimiter = '\t')
            for i, line in enumerate(reader):
              image_id_concepts_dict[line[0]+'.jpg'] = list(line[1].split(';'))

    return image_id_concepts_dict

def transform_images(training_images_dir, image):
  path_to_image = os.path.join(training_images_dir, image)
  img = tf.keras.preprocessing.image.load_img(path = path_to_image, target_size= (224,224))
  img = tf.keras.preprocessing.image.img_to_array(img)
  img = tf.keras.applications.densenet.preprocess_input(img)

  return img

In [ ]:
# Path and csv name to concepts file for training and validation images
path_to_concepts = ['/content/ImageCLEF2021_ConceptDetection_Training-Set/ImageCLEF2021_ConceptDetection_Training-Set',
                    '/content/ImageCLEF2021_ConceptDetection_Validation-Set']
concepts_file_name = ['Training_Set_Concepts.csv', 'Validation_Set_Concepts.csv']

#Extract concepts for the validation and training images and save to dict
image_id_concepts_dict = extract_concepts(path_to_concepts,concepts_file_name)


In [ ]:
# Define array where images array will be saved for training, and where the concepts for each image will be saved (Y)
X = []
Y = []

In [ ]:
# Training images
training_path = '/content/ImageCLEF2021_ConceptDetection_Training-Set/ImageCLEF2021_ConceptDetection_Training-Set/Training-Images'
for image in tqdm(os.listdir(training_path), position = 0):
  X.append(transform_images(training_path, image))
  Y.append(image_id_concepts_dict[image])

100%|██████████| 2756/2756 [00:20<00:00, 132.05it/s]


In [ ]:
3# Validation images images
validation_path = '/content/ImageCLEF2021_ConceptDetection_Validation-Set/Validation-Images'
for image in tqdm(os.listdir(validation_path), position = 0):
  X.append(transform_images(validation_path, image))
  Y.append(image_id_concepts_dict[image])

100%|██████████| 500/500 [00:03<00:00, 144.54it/s]


In [ ]:
# Transform arrays to numpy arrays
X = np.array(X)
Y = np.array(Y)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# Use a multilabelbinarizer to transform the concepts into a binary format for training
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(Y)
print(len(mlb.classes_))

1585


In [ ]:
# Fine tunning part
default_densenet.trainable = False # Set the densenet-121 layers to train = False before creating new model
print(len(default_densenet.trainable_variables))
#default_densenet.summary()

0


In [ ]:
x = default_densenet.layers[-2].output
prediction_layer = tf.keras.layers.Dense(len(mlb.classes_), activation= 'sigmoid')(x)
fine_tuned_model = tf.keras.models.Model(inputs = default_densenet.input, outputs = prediction_layer)
fine_tuned_model.summary()
print("Total number of trainable variables (layer weights + bias): ", len(fine_tuned_model.trainable_variables))

In [ ]:
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate, decay_steps=100, decay_rate=0.90, staircase=True)

fine_tuned_model.compile(loss = 'binary_crossentropy', optimizer= tf.keras.optimizers.Adam(learning_rate=lr_schedule), metrics = ['acc'])

callback_earlystop = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights= True)

In [ ]:
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(validation_split = 0.2)
train_generator = data_generator.flow(X,Y, batch_size = 32, subset = 'training', seed = 14)
val_generator = data_generator.flow(X,Y, batch_size = 32, subset = 'validation', seed = 14)

In [ ]:
history = fine_tuned_model.fit(train_generator, steps_per_epoch= 50, epochs = 15, validation_data= val_generator, validation_steps = 12, verbose= 1,
                               callbacks = [callback_earlystop])

Epoch 1/15
50/50 [==============================] - 45s 147ms/step - loss: 0.1769 - acc: 0.1288 - val_loss: 0.0109 - val_acc: 0.2969
Epoch 2/15
50/50 [==============================] - 6s 116ms/step - loss: 0.0115 - acc: 0.2929 - val_loss: 0.0100 - val_acc: 0.3229
Epoch 3/15
50/50 [==============================] - 7s 137ms/step - loss: 0.0109 - acc: 0.3163 - val_loss: 0.0100 - val_acc: 0.3646
Epoch 4/15
50/50 [==============================] - 6s 115ms/step - loss: 0.0103 - acc: 0.3447 - val_loss: 0.0093 - val_acc: 0.3516
Epoch 5/15
50/50 [==============================] - 6s 117ms/step - loss: 0.0100 - acc: 0.3315 - val_loss: 0.0092 - val_acc: 0.3542
Epoch 6/15
50/50 [==============================] - 6s 117ms/step - loss: 0.0095 - acc: 0.3347 - val_loss: 0.0097 - val_acc: 0.3490
Epoch 7/15
50/50 [==============================] - 6s 119ms/step - loss: 0.0094 - acc: 0.3687 - val_loss: 0.0095 - val_acc: 0.3698
Epoch 8/15
50/50 [==============================] - 6s 118ms/step - loss: 0

In [ ]:
default_densenet.trainable = True

In [ ]:
fine_tune_at = 277

for layer in default_densenet.layers[:fine_tune_at]:
  layer.trainable = False

In [ ]:
print(len(default_densenet.trainable_variables))

130


In [ ]:
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate, decay_steps=100, decay_rate=0.90, staircase=True)

fine_tuned_model.compile(loss='binary_crossentropy', optimizer= tf.keras.optimizers.Adam(learning_rate=lr_schedule), metrics = ['acc'])

In [ ]:
fine_tune_epochs = 20
total_epochs =  15 + fine_tune_epochs

history_fine = fine_tuned_model.fit(train_generator, steps_per_epoch= 50, epochs = total_epochs, validation_data= val_generator, validation_steps = 12, 
                                    verbose= 1, callbacks = [callback_earlystop], initial_epoch = history.epoch[-1])

Epoch 15/35
50/50 [==============================] - 17s 193ms/step - loss: 0.0084 - acc: 0.3462 - val_loss: 0.0107 - val_acc: 0.3854
Epoch 16/35
50/50 [==============================] - 8s 153ms/step - loss: 0.0068 - acc: 0.3773 - val_loss: 0.0088 - val_acc: 0.3854
Epoch 17/35
50/50 [==============================] - 8s 155ms/step - loss: 0.0055 - acc: 0.3757 - val_loss: 0.0078 - val_acc: 0.3828
Epoch 18/35
50/50 [==============================] - 8s 157ms/step - loss: 0.0044 - acc: 0.3831 - val_loss: 0.0079 - val_acc: 0.3750
Epoch 19/35
50/50 [==============================] - 8s 156ms/step - loss: 0.0037 - acc: 0.3390 - val_loss: 0.0071 - val_acc: 0.3646
Epoch 20/35
50/50 [==============================] - 8s 159ms/step - loss: 0.0029 - acc: 0.3762 - val_loss: 0.0075 - val_acc: 0.3776
Epoch 21/35
50/50 [==============================] - 8s 157ms/step - loss: 0.0023 - acc: 0.3738 - val_loss: 0.0070 - val_acc: 0.3828
Epoch 22/35
50/50 [==============================] - 8s 158ms/step -

In [ ]:
fine_tuned_model.save('fine-tuned-model-224-densenet121-2021.h5')

In [ ]:
# Load model when training is not needed
#fine_tuned_model = tf.keras.models.load_model('/content/fine-tuned-model-densenet121-2021.h5')
fine_tuned_model.summary()

In [ ]:
get_layer_output = tf.keras.backend.function([fine_tuned_model.layers[0].input],[fine_tuned_model.layers[-2].output])

In [ ]:
def feedForward_finetuned(fname,get_layer_output):

    img = tf.keras.preprocessing.image.load_img(fname, target_size=(224,224))
    x = tf.keras.preprocessing.image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    x = tf.keras.applications.densenet.preprocess_input(x)
    features = get_layer_output([x])[0]
    features = features.flatten()

    return features

def extract_store_finetuned(path_dir, model):
  """
    Function that takes a path and passes all the images (inside that directory) to the feedForward function for feature extraction, and then creates
    a dataframe containing all features and image ids of the images.
    path_dir: full path of the directory that will be searched
    model: model to be used for feature extraction

    Returns: A pandas DataFrame containing all images and their corresponding features
  """
  id_features_vector = []
  print('Extracting features...')

  for image in tqdm(os.listdir(path_dir), position= 0, leave = False):
    path = os.path.join(path_dir, image)
    name_image = os.path.splitext(image)[0]
    image_id = int(name_image.replace('synpic',''))
    vector = feedForward_finetuned(path, model)
    vector = np.insert(vector,0,image_id)
    id_features_vector.append(vector)


  #df_values = pd.DataFrame(id_features_vector, dtype = float)

  return id_features_vector

In [ ]:
# Training set
id_features_vector = extract_store_finetuned('/content/ImageCLEF2021_ConceptDetection_Training-Set/ImageCLEF2021_ConceptDetection_Training-Set/Training-Images', get_layer_output)
#default_features_df.to_csv('/content/features-densenet-fined-training-2021.csv', index = False, header=False)
np.save('/content/features-224-densenet-fined-training-2021.npy', id_features_vector)
# Validation set
id_features_vector = extract_store_finetuned('/content/ImageCLEF2021_ConceptDetection_Validation-Set/Validation-Images', get_layer_output)
#default_features_df.to_csv('/content/features-densenet-fined-validation-2021.csv', index = False, header=False)
np.save('/content/features-224-densenet-fined-validation-2021.npy', id_features_vector)

  0%|          | 1/2756 [00:00<05:48,  7.90it/s]

Extracting features...


  0%|          | 1/500 [00:00<01:09,  7.14it/s]

Extracting features...


## Modality classifier model (with weigths from previous training) wo fine-tunning

In [ ]:
# Load best model (modality classifier) and use as a feature extractor
modality_clf_base = tf.keras.models.load_model('/content/densenet121-modality-classifier-224-best.h5')

In [ ]:
modality_clf_base.summary()

In [ ]:
# This is to create a function that given a input, it returns the output of layer within a model. Layer[-6] is a BatchNormalization layer.
get_layer_output = tf.keras.backend.function([modality_clf_base.layers[0].input],[modality_clf_base.layers[-4].output])

In [ ]:
def feedForward_finetuned(fname,get_layer_output):

    img = tf.keras.preprocessing.image.load_img(fname, target_size=(224,224))
    x = tf.keras.preprocessing.image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    x = tf.keras.applications.densenet.preprocess_input(x)
    features = get_layer_output([x])[0]
    features = features.flatten()

    return features

def extract_store_finetuned(path_dir, model):
  """
    Function that takes a path and passes all the images (inside that directory) to the feedForward function for feature extraction, and then creates
    a dataframe containing all features and image ids of the images.
    path_dir: full path of the directory that will be searched
    model: model to be used for feature extraction

    Returns: A pandas DataFrame containing all images and their corresponding features
  """
  id_features_vector = []
  print('Extracting features...')

  for image in tqdm(os.listdir(path_dir), position= 0, leave = False):
    path = os.path.join(path_dir, image)
    name_image = os.path.splitext(image)[0]
    image_id = int(name_image.replace('synpic',''))
    vector = feedForward_finetuned(path, model)
    vector = np.insert(vector,0,image_id)
    id_features_vector.append(vector)


  #df_values = pd.DataFrame(id_features_vector, dtype = float)

  return id_features_vector

In [ ]:
# Training set
id_features_vector = extract_store_finetuned('/content/ImageCLEF2021_ConceptDetection_Training-Set/ImageCLEF2021_ConceptDetection_Training-Set/Training-Images', get_layer_output)
#default_features_df.to_csv('/content/features-densenet-modality-clf-training-2021.csv', index = False, header=False)
np.save('/content/features-224-densenet-modality-clf-training-2021.npy', id_features_vector)
# Validation set
id_features_vector = extract_store_finetuned('/content/ImageCLEF2021_ConceptDetection_Validation-Set/Validation-Images', get_layer_output)
np.save('/content/features-224-densenet-modality-clf-validation-2021.npy', id_features_vector)
#default_features_df.to_csv('/content/features-densenet-modalityclf-validation-2021.csv', index = False, header=False)

  0%|          | 0/2756 [00:00<?, ?it/s]

Extracting features...


  0%|          | 1/500 [00:00<01:01,  8.09it/s]

Extracting features...


## Modality classifier model (with weigths from previous training) with fine-tunning

In [ ]:
# Load best model (modality classifier) and use as a feature extractor
modality_clf_base_fine = tf.keras.models.load_model('/content/densenet121-modality-classifier-224-best.h5')
modality_clf_base_fine.trainable = False
modality_clf_base_fine.summary()

In [ ]:
model2 = tf.keras.models.Model(inputs = modality_clf_base_fine.input, outputs = modality_clf_base_fine.layers[-2].output)

In [ ]:
# Utils functions
def extract_concepts(root_paths, csv_names, image_id_concepts_dict = dict()):

    if len(root_paths)==len(csv_names):
      for idx, name in enumerate(root_paths):
        path = os.path.join(name, csv_names[idx])
        if csv_names[idx] == 'Training_Set_Concepts.csv': # Make sure that you just convert the .xlsx file to csv manually
          with open(path, "r", encoding= 'utf-8-sig') as f:
            reader = csv.reader(f, delimiter = ',')
            for i, line in enumerate(reader):
              image_id_concepts_dict[line[0]+'.jpg'] = list(line[1].split(';'))
        else:
           with open(path, "r", encoding= 'utf-8-sig') as f:
            reader = csv.reader(f, delimiter = '\t')
            for i, line in enumerate(reader):
              image_id_concepts_dict[line[0]+'.jpg'] = list(line[1].split(';'))

    return image_id_concepts_dict

def transform_images(training_images_dir, image):
  path_to_image = os.path.join(training_images_dir, image)
  img = tf.keras.preprocessing.image.load_img(path = path_to_image, target_size= (64,64))
  img = tf.keras.preprocessing.image.img_to_array(img)
  img = tf.keras.applications.densenet.preprocess_input(img)

  return img

In [ ]:
# Path and csv name to concepts file for training and validation images
path_to_concepts = ['/content/ImageCLEF2021_ConceptDetection_Training-Set/ImageCLEF2021_ConceptDetection_Training-Set',
                    '/content/ImageCLEF2021_ConceptDetection_Validation-Set']
concepts_file_name = ['Training_Set_Concepts.csv', 'Validation_Set_Concepts.csv']

#Extract concepts for the validation and training images and save to dict
image_id_concepts_dict = extract_concepts(path_to_concepts,concepts_file_name)

In [ ]:
# Define array where images array will be saved for training, and where the concepts for each image will be saved (Y)
X = []
Y = []

In [ ]:
# Training images
training_path = '/content/ImageCLEF2021_ConceptDetection_Training-Set/ImageCLEF2021_ConceptDetection_Training-Set/Training-Images'
for image in tqdm(os.listdir(training_path), position = 0):
  X.append(transform_images(training_path, image))
  Y.append(image_id_concepts_dict[image])

100%|██████████| 2756/2756 [00:18<00:00, 147.21it/s]


In [ ]:
# Validation images images
validation_path = '/content/ImageCLEF2021_ConceptDetection_Validation-Set/Validation-Images'
for image in tqdm(os.listdir(validation_path), position = 0):
  X.append(transform_images(validation_path, image))
  Y.append(image_id_concepts_dict[image])

100%|██████████| 500/500 [00:03<00:00, 162.76it/s]


In [ ]:
# Transform arrays to numpy arrays
X = np.array(X)
Y = np.array(Y)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# Use a multilabelbinarizer to transform the concepts into a binary format for training
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(Y)
print(len(mlb.classes_))

1585


In [ ]:
x = model2.output
prediction_layer = tf.keras.layers.Dense(len(mlb.classes_), activation= 'sigmoid')(x)
fine_tuned_model = tf.keras.models.Model(inputs = model2.input, outputs = prediction_layer)
print("Total number of trainable variables (layer weights + bias): ", len(fine_tuned_model.trainable_variables))

Total number of trainable variables (layer weights + bias):  2


In [ ]:
fine_tuned_model.summary()

In [ ]:
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate, decay_steps=100, decay_rate=0.90, staircase=True)

fine_tuned_model.compile(loss = 'binary_crossentropy', optimizer= tf.keras.optimizers.Adam(learning_rate=lr_schedule), metrics = ['acc'])

callback_earlystop = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights= True)

In [ ]:
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(validation_split = 0.2)
train_generator = data_generator.flow(X,Y, batch_size = 32, subset = 'training', seed = 14)
val_generator = data_generator.flow(X,Y, batch_size = 32, subset = 'validation', seed = 14)

In [ ]:
history = fine_tuned_model.fit(train_generator, steps_per_epoch= 50, epochs = 15, validation_data= val_generator, validation_steps = 12, verbose= 1,
                               callbacks = [callback_earlystop])

Epoch 1/15
50/50 [==============================] - 41s 68ms/step - loss: 0.2150 - acc: 0.1272 - val_loss: 0.0139 - val_acc: 0.2708
Epoch 2/15
50/50 [==============================] - 1s 22ms/step - loss: 0.0147 - acc: 0.2465 - val_loss: 0.0122 - val_acc: 0.2656
Epoch 3/15
50/50 [==============================] - 1s 22ms/step - loss: 0.0131 - acc: 0.2676 - val_loss: 0.0122 - val_acc: 0.3307
Epoch 4/15
50/50 [==============================] - 1s 22ms/step - loss: 0.0125 - acc: 0.2864 - val_loss: 0.0114 - val_acc: 0.2708
Epoch 5/15
50/50 [==============================] - 1s 22ms/step - loss: 0.0118 - acc: 0.2786 - val_loss: 0.0110 - val_acc: 0.3151
Epoch 6/15
50/50 [==============================] - 1s 22ms/step - loss: 0.0114 - acc: 0.3040 - val_loss: 0.0115 - val_acc: 0.2995
Epoch 7/15
50/50 [==============================] - 1s 21ms/step - loss: 0.0110 - acc: 0.2806 - val_loss: 0.0114 - val_acc: 0.2839
Epoch 8/15
50/50 [==============================] - 1s 22ms/step - loss: 0.0111 - 

In [ ]:
model2.trainable = True

In [ ]:
fine_tune_at = 277

for layer in model2.layers[:fine_tune_at]:
  layer.trainable = False

In [ ]:
fine_tuned_model.summary()

In [ ]:
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate, decay_steps=100, decay_rate=0.90, staircase=True)

fine_tuned_model.compile(loss = 'binary_crossentropy', optimizer= tf.keras.optimizers.Adam(learning_rate=lr_schedule), metrics = ['acc'])

In [ ]:
fine_tune_epochs = 50
total_epochs =  15 + fine_tune_epochs

history_fine = fine_tuned_model.fit(train_generator, steps_per_epoch= 50, epochs = total_epochs, validation_data= val_generator, validation_steps = 12, 
                                    verbose= 1, callbacks = [callback_earlystop], initial_epoch = history.epoch[-1])

Epoch 15/65
50/50 [==============================] - 11s 75ms/step - loss: 0.0127 - acc: 0.2405 - val_loss: 0.0111 - val_acc: 0.2630
Epoch 16/65
50/50 [==============================] - 2s 36ms/step - loss: 0.0090 - acc: 0.3304 - val_loss: 0.0103 - val_acc: 0.3177
Epoch 17/65
50/50 [==============================] - 2s 37ms/step - loss: 0.0078 - acc: 0.3452 - val_loss: 0.0092 - val_acc: 0.3359
Epoch 18/65
50/50 [==============================] - 2s 37ms/step - loss: 0.0071 - acc: 0.3550 - val_loss: 0.0091 - val_acc: 0.3385
Epoch 19/65
50/50 [==============================] - 2s 37ms/step - loss: 0.0061 - acc: 0.3528 - val_loss: 0.0083 - val_acc: 0.3307
Epoch 20/65
50/50 [==============================] - 2s 36ms/step - loss: 0.0052 - acc: 0.3668 - val_loss: 0.0088 - val_acc: 0.3594
Epoch 21/65
50/50 [==============================] - 2s 36ms/step - loss: 0.0047 - acc: 0.3575 - val_loss: 0.0081 - val_acc: 0.3698
Epoch 22/65
50/50 [==============================] - 2s 36ms/step - loss: 0

In [ ]:
fine_tuned_model.save('fine-tuned-224-modality-clf-densenet121-2021.h5')

In [ ]:
fine_tuned_model = tf.keras.models.load_model('/content/fine-tuned-224-modality-clf-densenet121-2021.h5')
fine_tuned_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_8 (ZeroPadding2D (None, 230, 230, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_8[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
____________________________________________________________________________________________

In [ ]:
get_layer_output = tf.keras.backend.function([fine_tuned_model.layers[0].input],[fine_tuned_model.layers[-4].output])

In [ ]:
def feedForward_finetuned(fname,get_layer_output):

    img = tf.keras.preprocessing.image.load_img(fname, target_size=(224,224))
    x = tf.keras.preprocessing.image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    x = tf.keras.applications.densenet.preprocess_input(x)
    features = get_layer_output([x])[0]
    features = features.flatten()

    return features

def extract_store_finetuned(path_dir, model):
  """
    Function that takes a path and passes all the images (inside that directory) to the feedForward function for feature extraction, and then creates
    a dataframe containing all features and image ids of the images.
    path_dir: full path of the directory that will be searched
    model: model to be used for feature extraction

    Returns: A pandas DataFrame containing all images and their corresponding features
  """
  id_features_vector = []
  print('Extracting features...')

  for image in tqdm(os.listdir(path_dir), position= 0, leave = False):
    path = os.path.join(path_dir, image)
    name_image = os.path.splitext(image)[0]
    image_id = int(name_image.replace('synpic',''))
    vector = feedForward_finetuned(path, model)
    vector = np.insert(vector,0,image_id)
    id_features_vector.append(vector)


  #df_values = pd.DataFrame(id_features_vector, dtype = float)

  return id_features_vector

In [ ]:
# Training set
id_features_vector = extract_store_finetuned('/content/ImageCLEF2021_ConceptDetection_Training-Set/ImageCLEF2021_ConceptDetection_Training-Set/Training-Images', get_layer_output)
#default_features_df.to_csv('/content/features-densenet-modality-clf-fined-training-2021.csv', index = False, header=False)
np.save('/content/features-224-densenet-modality-clf-fined-training-2021.npy', id_features_vector)
# Validation set
id_features_vector = extract_store_finetuned('/content/ImageCLEF2021_ConceptDetection_Validation-Set/Validation-Images', get_layer_output)
#default_features_df.to_csv('/content/features-densenet-modality-clf-fined-validation-2021.csv', index = False, header=False)
np.save('/content/features-224-densenet-modality-clf-fined-validation-2021.npy', id_features_vector)

  0%|          | 0/2756 [00:00<?, ?it/s]

Extracting features...


  0%|          | 1/500 [00:00<01:06,  7.51it/s]

Extracting features...


In [ ]:
# Utils functions
def extract_concepts(root_paths, image_id_concepts_dict = dict()):

    for idx, name in enumerate(root_paths):
      with open(name, "r", encoding= 'utf-8-sig') as f:
        reader = csv.reader(f, delimiter = '\t')
        
        if name == '/content/Training_Set_Concepts.csv':
          image_path = '/content/ImageCLEF2021_ConceptDetection_Training-Set/ImageCLEF2021_ConceptDetection_Training-Set/Training-Images/'
        else:
          image_path = '/content/ImageCLEF2021_ConceptDetection_Validation-Set/Validation-Images/'
        for i, line in enumerate(reader):
          if len(line[1]) < 1:
            image_id_concepts_dict[image_path+line[0]+'.jpg'] = []
          else:
            image_id_concepts_dict[image_path+line[0]+'.jpg'] = list(line[1].split(';'))

    return image_id_concepts_dict

In [ ]:
# Path and csv name to concepts file for training and validation images
path_to_concepts = ['/content/Training_Set_Concepts.csv','/content/ImageCLEF2021_ConceptDetection_Validation-Set/Validation_Set_Concepts.csv']

#Extract concepts for the validation and training images and save to dict
image_id_concepts_dict = extract_concepts(path_to_concepts)

In [ ]:
# Since we are working with around 9K images. We will only load the images absolute path and the concepts to a dataframe and then use a generator to load them during training.
# Here, we will create a dataframe with the images path
all_images_path = []
# Training images
for image in tqdm(image_id_concepts_dict.keys(), position = 0):
  all_images_path.append([image])
df_all_images = pd.DataFrame(all_images_path, columns=['image_path'])

100%|██████████| 3256/3256 [00:00<00:00, 898878.02it/s]


In [ ]:
concepts = []
for image in tqdm(image_id_concepts_dict.keys(), position=0):
  concepts.append(image_id_concepts_dict[image])

100%|██████████| 3256/3256 [00:00<00:00, 1044166.51it/s]


In [ ]:
mlb = MultiLabelBinarizer()

In [ ]:
# Since we will use flow_from_dataframe in the training, we put both the images absolute path and the encoded labels
df_use = pd.concat([df_all_images, pd.DataFrame(np.array(mlb.fit_transform(concepts)))], axis=1)

In [ ]:
default_efficient_b4 = tf.keras.applications.EfficientNetB4(include_top=False, input_shape = (492,492,3))

71688192/71686520 [==============================] - 0s 0us/step


In [ ]:
default_efficient_b4.summary()

In [ ]:
default_efficient_b4.trainable = False

In [ ]:
x = tf.keras.layers.GlobalAveragePooling2D()(default_efficient_b4.output)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(1585, activation='sigmoid')(x)

custom_efficient_b4 = tf.keras.models.Model(inputs = default_efficient_b4.input, outputs = x)

In [ ]:
custom_efficient_b4.summary()

In [ ]:
# Define some required parameter for training
init_lr = 1e-4
epochs = 100
batch_size = 32
valid_batch_size = 32

# Objects to be used by the model
opt = tf.keras.optimizers.Adam(lr=init_lr, decay=init_lr / epochs)
callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_acc', patience = 5, restore_best_weights= True, mode = 'max')]

In [ ]:
# Compile model
custom_efficient_b4.compile(loss = 'binary_crossentropy', optimizer=opt, metrics=['acc'])

In [ ]:
# Data generator
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(validation_split = 0.2, rescale=1./255) # This will split the training dataframe, and also rescale the values from loaded images

# Train generator
train_generator = data_generator.flow_from_dataframe(df_use,x_col='image_path', y_col=df_use.columns[1:], target_size=(492,492),
                                                     class_mode ='raw',batch_size=32, shuffle=True, seed=14, subset='training')

# Validation generator
val_generator = data_generator.flow_from_dataframe(df_use,x_col='image_path', y_col=df_use.columns[1:], target_size=(492,492),
                                                     class_mode ='raw',batch_size=32, shuffle=True, seed=14, subset='validation')

Found 2605 validated image filenames.
Found 651 validated image filenames.


In [ ]:
# Model training (only the classification layers that have been added)
history = custom_efficient_b4.fit(train_generator, epochs = epochs, validation_data= val_generator, verbose= 1,
                               callbacks = callbacks, batch_size = 32)

In [ ]:
layer_names = [layer.name for layer in default_efficient_b4.layers]

In [ ]:
layer_idx = layer_names.index('block5a_expand_conv')

In [ ]:
for layer in default_efficient_b4.layers[layer_idx:]:
  layer.trainable = True

In [ ]:
# Define some required parameter for training
init_lr = 1e-5
epochs = 100

# Objects to be used by the model
opt = tf.keras.optimizers.Adam(lr=init_lr, decay=init_lr / epochs)
callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_acc', patience = 5, restore_best_weights= True, mode = 'max')]

In [ ]:
# Compile model
custom_efficient_b4.compile(loss = 'binary_crossentropy', optimizer=opt, metrics=['acc'])

In [ ]:
# Model training (only the classification layers that have been added)
history_fined = custom_efficient_b4.fit(train_generator, epochs = epochs, validation_data= val_generator, verbose= 1,
                               callbacks = callbacks, batch_size = 32)

Epoch 1/100
82/82 [==============================] - 150s 2s/step - loss: 0.8188 - acc: 0.0128 - val_loss: 0.2068 - val_acc: 0.1674
Epoch 2/100
82/82 [==============================] - 133s 2s/step - loss: 0.2231 - acc: 0.1659 - val_loss: 0.1266 - val_acc: 0.1674
Epoch 3/100
82/82 [==============================] - 133s 2s/step - loss: 0.1088 - acc: 0.1713 - val_loss: 0.0691 - val_acc: 0.1674
Epoch 4/100
82/82 [==============================] - 133s 2s/step - loss: 0.0743 - acc: 0.1736 - val_loss: 0.0536 - val_acc: 0.1674
Epoch 5/100
82/82 [==============================] - 133s 2s/step - loss: 0.0563 - acc: 0.1808 - val_loss: 0.0463 - val_acc: 0.1674
Epoch 6/100
82/82 [==============================] - 133s 2s/step - loss: 0.0462 - acc: 0.1715 - val_loss: 0.0404 - val_acc: 0.1674


In [ ]:
get_layer_output = tf.keras.backend.function([custom_efficient_b4.layers[0].input],[custom_efficient_b4.layers[-3].output])

In [ ]:
def feedForward_finetuned(fname,get_layer_output):

    img = tf.keras.preprocessing.image.load_img(fname, target_size=(492,492))
    x = tf.keras.preprocessing.image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    x = tf.keras.applications.efficientnet.preprocess_input(x)
    features = get_layer_output([x])[0]
    features = features.flatten()

    return features

def extract_store_finetuned(path_dir, model):
  """
    Function that takes a path and passes all the images (inside that directory) to the feedForward function for feature extraction, and then creates
    a dataframe containing all features and image ids of the images.
    path_dir: full path of the directory that will be searched
    model: model to be used for feature extraction

    Returns: A pandas DataFrame containing all images and their corresponding features
  """
  id_features_vector = []
  print('Extracting features...')

  for image in tqdm(os.listdir(path_dir), position= 0, leave = False):
    path = os.path.join(path_dir, image)
    name_image = os.path.splitext(image)[0]
    image_id = int(name_image.replace('synpic',''))
    vector = feedForward_finetuned(path, model)
    vector = np.insert(vector,0,image_id)
    id_features_vector.append(vector)


  #df_values = pd.DataFrame(id_features_vector, dtype = float)

  return id_features_vector

In [ ]:
# Training set
id_features_vector = extract_store_finetuned('/content/ImageCLEF2021_ConceptDetection_Training-Set/ImageCLEF2021_ConceptDetection_Training-Set/Training-Images', get_layer_output)
np.save('/content/features-492-efficient4-fined-training-2021.npy', id_features_vector)

# Validation set
id_features_vector = extract_store_finetuned('/content/ImageCLEF2021_ConceptDetection_Validation-Set/Validation-Images', get_layer_output)

np.save('/content/features-492-efficient-fined-validation-2021.npy', id_features_vector)

  0%|          | 1/2756 [00:00<07:14,  6.34it/s]

Extracting features...


  0%|          | 1/500 [00:00<01:10,  7.12it/s]

Extracting features...


In [ ]:
custom_efficient_b4.save('fine-tuned-efficient4.h5')

# Feature extraction using DenseNet-121 and 2021+2020 dataset

## Default Densenet-121 model (initialized with imagenet weights) with fine-tunning

In [ ]:
# Function to extract concepts from the training and validation images from 2021, and also from selected images from 2020 ImageCLEF dataset
# These selected images are images that strictly have the same concepts of this year dataset, therefore, 6,556 images from last year dataset are being used
def extract_concepts_all(root_paths, image_id_concepts_dict = dict()):
    """
      Function that extract concepts for a concept file (csv and json), and stores them in a dictionary, 
      where the key is the absolute path of the image and the values are the concepts.

      root_paths: a 1-d list that contains the absolute paths of the concept files
      image_id_concepts_dict: dictionary that will contain the data from the concepts file

      Returns: a dictionary with the absolute images paths as keys and their corresponding concepts as values.

    """
    for idx, name in enumerate(root_paths):
      if idx < 2:
        with open(name, "r", encoding= 'utf-8-sig') as f:
          reader = csv.reader(f, delimiter = '\t')
          if name =='/content/Training_Set_Concepts.csv':
            path_image = '/content/ImageCLEF2021_ConceptDetection_Training-Set/ImageCLEF2021_ConceptDetection_Training-Set/Training-Images/'

          if name == '/content/ImageCLEF2021_ConceptDetection_Validation-Set/Validation_Set_Concepts.csv':
            path_image = '/content/ImageCLEF2021_ConceptDetection_Validation-Set/Validation-Images/'

          for i, line in enumerate(reader):
    
              if len(line[1]) < 1:
                image_id_concepts_dict[path_image+line[0]+'.jpg'] = []
              else:
                image_id_concepts_dict[path_image+line[0]+'.jpg'] = list(line[1].split(';'))
                
      else:
        # This section is strictly for the images from 2020 dataset
        for concept_file in os.listdir(name):
          modality = concept_file.split("_")[0]
          if name == '/content/ImageCLEF2020_Train_Concepts':
            path_image = '/content/ImageCLEF2020_Train_Images/Train/%s/' %modality
          if name == '/content/ImageCLEF2020_Validation_Concepts':
            path_image = '/content/ImageCLEF2020_Validation_Images/Validation/%s/' %modality

          with open(os.path.join(name,concept_file), "r", encoding='utf-8-sig') as f:
            reader = csv.reader(f)
            for i, line in enumerate(reader):
              image_id_concepts_dict[path_image+line[0]] = line[1:]

    return image_id_concepts_dict

In [ ]:
path_to_concepts = ['/content/Training_Set_Concepts.csv',
                    '/content/ImageCLEF2021_ConceptDetection_Validation-Set/Validation_Set_Concepts.csv',
                    '/content/ImageCLEF2020_Train_Concepts','/content/ImageCLEF2020_Validation_Concepts']
image_id_concepts_dict = extract_concepts_all(path_to_concepts)

In [ ]:
images_path_and_concepts = []
for image in image_id_concepts_dict.keys():
  images_path_and_concepts.append([image,image_id_concepts_dict[image]])

df_images_concepts = pd.DataFrame(images_path_and_concepts, columns=['image_path', 'tags'])

In [ ]:
default_densenet = tf.keras.applications.densenet.DenseNet121(include_top=True, weights= 'imagenet')

In [ ]:
default_densenet.trainable = False # Set the densenet-121 layers to train = False before creating new mode

In [ ]:
x = default_densenet.layers[-2].output
prediction_layer = tf.keras.layers.Dense(4339, activation= 'sigmoid')(x)
fine_tuned_model = tf.keras.models.Model(inputs = default_densenet.input, outputs = prediction_layer)

In [ ]:
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate, decay_steps=100, decay_rate=0.90, staircase=True)

fine_tuned_model.compile(loss = 'binary_crossentropy', optimizer= tf.keras.optimizers.Adam(learning_rate=lr_schedule), metrics = ['acc'])

callback_earlystop = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights= True)

In [ ]:
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(validation_split = 0.2, rescale=1./255)

In [ ]:
train_generator = data_generator.flow_from_dataframe(df_images_concepts, x_col='image_path',y_col='tags',target_size=(224,224), 
                                                     class_mode='categorical',shuffle=True, seed=14, subset = 'training')
val_generator = data_generator.flow_from_dataframe(df_images_concepts, x_col='image_path',y_col='tags',target_size=(224,224), 
                                                     class_mode='categorical',shuffle=True, seed=14, subset = 'validation')

Found 67184 validated image filenames belonging to 4339 classes.
Found 16795 validated image filenames belonging to 4339 classes.


In [ ]:
history = fine_tuned_model.fit(train_generator, epochs = 15, validation_data= val_generator, verbose= 1,
                               callbacks = [callback_earlystop])

Epoch 1/15
2100/2100 [==============================] - 538s 238ms/step - loss: 0.0237 - acc: 0.2496 - val_loss: 0.0124 - val_acc: 0.2732
Epoch 2/15
2100/2100 [==============================] - 469s 223ms/step - loss: 0.0137 - acc: 0.2860 - val_loss: 0.0123 - val_acc: 0.2760
Epoch 3/15
2100/2100 [==============================] - 452s 215ms/step - loss: 0.0135 - acc: 0.2859 - val_loss: 0.0123 - val_acc: 0.2762
Epoch 4/15
2100/2100 [==============================] - 452s 215ms/step - loss: 0.0134 - acc: 0.2880 - val_loss: 0.0123 - val_acc: 0.2762
Epoch 5/15
2100/2100 [==============================] - 454s 216ms/step - loss: 0.0135 - acc: 0.2862 - val_loss: 0.0123 - val_acc: 0.2762
Epoch 6/15
2100/2100 [==============================] - 454s 216ms/step - loss: 0.0134 - acc: 0.2911 - val_loss: 0.0123 - val_acc: 0.2762
Epoch 7/15
2100/2100 [==============================] - 455s 217ms/step - loss: 0.0133 - acc: 0.2913 - val_loss: 0.0123 - val_acc: 0.2762
Epoch 8/15
2100/2100 [============

In [ ]:
default_densenet.trainable = True

In [ ]:
initial_learning_rate = 0.0001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate, decay_steps=100, decay_rate=0.90, staircase=True)

fine_tuned_model.compile(loss='binary_crossentropy', optimizer= tf.keras.optimizers.Adam(learning_rate=lr_schedule), metrics = ['acc'])

In [ ]:
fine_tune_epochs = 100
total_epochs =  15 + fine_tune_epochs

history_fine = fine_tuned_model.fit(train_generator, epochs = total_epochs, validation_data= val_generator, 
                                    verbose= 1, callbacks = [callback_earlystop], initial_epoch = history.epoch[-1])

Epoch 9/115
2100/2100 [==============================] - 871s 409ms/step - loss: 0.0137 - acc: 0.3022 - val_loss: 0.0120 - val_acc: 0.2807
Epoch 10/115
2100/2100 [==============================] - 857s 408ms/step - loss: 0.0132 - acc: 0.2908 - val_loss: 0.0120 - val_acc: 0.2797
Epoch 11/115
2100/2100 [==============================] - 859s 409ms/step - loss: 0.0133 - acc: 0.2887 - val_loss: 0.0120 - val_acc: 0.2787
Epoch 12/115
2100/2100 [==============================] - 858s 409ms/step - loss: 0.0132 - acc: 0.2887 - val_loss: 0.0120 - val_acc: 0.2797
Epoch 13/115
2100/2100 [==============================] - 860s 409ms/step - loss: 0.0132 - acc: 0.2902 - val_loss: 0.0120 - val_acc: 0.2788
Epoch 14/115
2100/2100 [==============================] - 860s 409ms/step - loss: 0.0132 - acc: 0.2914 - val_loss: 0.0120 - val_acc: 0.2786
Epoch 15/115
2100/2100 [==============================] - 860s 409ms/step - loss: 0.0132 - acc: 0.2911 - val_loss: 0.0120 - val_acc: 0.2794
Epoch 16/115
2100/210

# Kaggle API commands to create dataset

In [ ]:
!kaggle datasets init -p image-features-2020-2021
# Change metadata file


Data package template written to: image-features-2020-2021/dataset-metadata.json
Default slug detected, please change values before uploading


In [ ]:
!kaggle datasets create -p image-features-2020-2021

Starting upload for file train_concepts_2020_2021_sample_merged.csv
100% 2.85M/2.85M [00:00<00:00, 6.43MB/s]
Upload successful: train_concepts_2020_2021_sample_merged.csv (3MB)
Skipping folder: .ipynb_checkpoints; use '--dir-mode' to upload folders
Starting upload for file features-images-2020-2021-sample-default-densenet-shard-1.npy
100% 2.43G/2.43G [01:12<00:00, 36.0MB/s]
Upload successful: features-images-2020-2021-sample-default-densenet-shard-1.npy (2GB)
Starting upload for file features-images-2020-2021-sample-default-densenet-shard-2.npy
100% 2.61G/2.61G [01:17<00:00, 36.1MB/s]
Upload successful: features-images-2020-2021-sample-default-densenet-shard-2.npy (3GB)
Your private Dataset is being created. Please check progress at https://www.kaggle.com/fjpa121197/image-features-2020-2021
